In [17]:
%config InlineBackend.figure_format = "svg"

# Import models and data splitting from sktime
from sktime.forecasting.compose import ForecastingPipeline, StackingForecaster
from sktime.transformations.series.func_transform import FunctionTransformer
from sktime.transformations.series.adapt import PandasTransformAdaptor
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sktime.forecasting.naive import NaiveForecaster

# Import metrics
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError

# Imports for defining a custom sklearn regressor
from sklearn.base import BaseEstimator, RegressorMixin

# Series decomposition
from statsmodels.tsa.seasonal import seasonal_decompose

# Silence warnings
import warnings
warnings.filterwarnings('ignore')

# Data acquisition, processing and visualization tools
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np

## Loading data

In [2]:
datapath = "data/train.csv"
df = pd.read_csv(datapath, index_col="date")
df = df.set_index(pd.PeriodIndex(df.index, freq="D").to_timestamp()).to_period("D")
df

,tavg,tmin,tmax,wdir,wspd,pres
date,,,,,,
2018-01-01,3.1,0.6,5.4,48.0,6.5,1011.7
2018-01-02,5.0,2.3,7.7,335.0,11.4,1009.5
2018-01-03,4.0,0.7,6.6,223.0,12.2,1007.9
2018-01-04,4.7,2.2,10.4,190.0,8.0,1005.8
2018-01-05,6.4,3.7,9.5,144.0,6.3,1007.3
...,...,...,...,...,...,...
2022-09-23,12.0,6.8,17.3,300.0,5.3,1020.6
2022-09-24,12.2,5.5,18.2,98.0,8.3,1017.7
2022-09-25,13.7,11.0,16.5,73.0,6.1,1014.5


## Creating data transformers

As we saw already in the exploratory data analysis notebook, there are two steps needed for preprocessing our data:
- Linear interpolation for handling missing values
- Classical decomposition using `statsmodels`

Unfortunately, these processes are not natively supported by `sktime`, and therefore we will build our custom transformers.

For the Linear Interpolator, we can adapt the pandas interpolate method into a transformer using the `PandasTransformAdaptor`:

In [3]:
interpolator = PandasTransformAdaptor(method="interpolate", kwargs={"method": "linear"})
inter_df = interpolator.fit_transform(df)
inter_df.isna().sum()

tavg    0
tmin    0
tmax    0
wdir    0
wspd    0
pres    0
dtype: int64

To do this for the classical decomposition, we can simply use the `FunctionTransformer`, which requires the definition of the transformation functions:

In [4]:
def trend_decomposition(X, model, period, extrapolate_trend):
    # Create new df
    X = X.copy()
    
    # Iterate every column to transform
    for key in X:
        # Decompose series
        decomposed = seasonal_decompose(X[key], model=model, period=period, extrapolate_trend=extrapolate_trend)
        
        # Add dataframe trend and seasonal entries
        X[key] = decomposed.trend
        
    return X

def seasonal_decomposition(X, model, period, extrapolate_trend):
    # Create new df
    X = X.copy()
    
    # Iterate every column to transform
    for key in X:
        # Decompose series
        decomposed = seasonal_decompose(X[key], model=model, period=period, extrapolate_trend=extrapolate_trend)
        
        # Add dataframe trend and seasonal entries
        X[key] = decomposed.seasonal
        
    return X

Now let's create the decomposer:

In [5]:
# Decomposition arguments
model = "additive"
period = 365
extrapolate_trend = "freq"

# Create classical time-series decomposer
foo = lambda X: trend_decomposition(X, model, period, extrapolate_trend)
trend_decomposer = FunctionTransformer(func=foo, check_inverse=False)
trend_decomposer.fit_transform(inter_df)

,tavg,tmin,tmax,wdir,wspd,pres
date,,,,,,
2018-01-01,13.273631,9.213128,17.793790,162.989534,7.994651,1017.023757
2018-01-02,13.271941,9.210267,17.792590,163.066633,7.996125,1017.024462
2018-01-03,13.270252,9.207407,17.791391,163.143732,7.997598,1017.025168
2018-01-04,13.268562,9.204546,17.790192,163.220831,7.999071,1017.025873
2018-01-05,13.266873,9.201685,17.788993,163.297930,8.000545,1017.026579
...,...,...,...,...,...,...
2022-09-23,13.412273,8.797833,17.937123,211.594398,10.290384,1019.449024
2022-09-24,13.414180,8.799041,17.939825,211.558237,10.291902,1019.452468
2022-09-25,13.416086,8.800250,17.942528,211.522077,10.293421,1019.455911


In [6]:
# Create classical time-series decomposer
foo = lambda X: seasonal_decomposition(X, model, period, extrapolate_trend)
seasonal_decomposer = FunctionTransformer(func=foo, check_inverse=False)
seasonal_decomposer.fit_transform(inter_df)

,tavg,tmin,tmax,wdir,wspd,pres
date,,,,,,
2018-01-01,-8.725300,-7.469399,-9.725193,40.200566,-1.284147,3.020367
2018-01-02,-8.047264,-6.732608,-9.726542,68.397201,3.766736,1.459130
2018-01-03,-9.669008,-8.154940,-11.928987,2.355480,2.225839,2.977619
2018-01-04,-8.891410,-7.337546,-9.650775,34.485265,2.262476,0.075999
2018-01-05,-8.152113,-6.378672,-10.230590,-3.966319,3.356647,-2.425951
...,...,...,...,...,...,...
2022-09-23,2.170725,1.872893,2.521811,-6.555201,0.276405,-0.528248
2022-09-24,2.227001,2.211391,2.536065,43.592305,0.395718,-0.476498
2022-09-25,2.543551,3.110875,2.711525,4.604468,-2.163709,-0.245078


Hurray! Our decomposers works accordingly!

## Creating models and data pipeline

For this forecasting problem, we will be using the naive method for both trend and seasonal forecasting:
- Naive method for trend forecasting forecasts using the previous value of the trend
- Naive seasonal method forecasts using the corresponding value of the previous period

In [7]:
# Create trended forecaster
trend_forecaster = NaiveForecaster(strategy="last")

# Create trend pipeline
trend_pipeline = ForecastingPipeline(steps=[
    ("interpolator", interpolator),
    ("trend_decomposer", trend_decomposer),
    ("trend_forecaster", trend_forecaster)
])
trend_pipeline

ForecastingPipeline(steps=[('interpolator',
                            PandasTransformAdaptor(kwargs={'method': 'linear'},
                                                   method='interpolate')),
                           ('trend_decomposer',
                            FunctionTransformer(check_inverse=False,
                                                func=<function <lambda> at 0x7f725768fc70>)),
                           ('trend_forecaster', NaiveForecaster())])

In [9]:
# Create seasonal forecaster
seasonal_forecaster = NaiveForecaster(strategy="last", sp=365)

# Create seasonal pipeline
seasonal_pipeline = ForecastingPipeline(steps=[
    ("interpolator", interpolator),
    ("seasonal_decomposer", seasonal_decomposer),
    ("seasonal_forecaster", seasonal_forecaster)
])
seasonal_pipeline

ForecastingPipeline(steps=[('interpolator',
                            PandasTransformAdaptor(kwargs={'method': 'linear'},
                                                   method='interpolate')),
                           ('seasonal_decomposer',
                            FunctionTransformer(check_inverse=False,
                                                func=<function <lambda> at 0x7f725768f910>)),
                           ('seasonal_forecaster', NaiveForecaster(sp=365))])

## Joining the two pipelines

Now we have a pipeline for both the trend and seasonal forecasters. We want to join the two simply by summing their values to create a full pipeline.
We can do this by using the `sktime` `StackingForecaster`. An `sklear` regressor can be passed to join their outputs! As such, we will create a custom regressor that simply sums features!

In [10]:
class SumRegressor(BaseEstimator, RegressorMixin):
    
    def fit(self, X, y):
        return self
    
    def predict(self, X):
        return X.sum(axis=-1)

Now let's join the two pipelines:

In [12]:
# Create the regressor
regressor = SumRegressor()

pipeline = StackingForecaster(forecasters=[
        ("trend_forecaster", trend_pipeline),
        ("seasonal_forecaster", seasonal_pipeline)
    ], 
    regressor=regressor
)
pipeline

StackingForecaster(forecasters=[('trend_forecaster',
                                 ForecastingPipeline(steps=[('interpolator',
                                                             PandasTransformAdaptor(kwargs={'method': 'linear'},
                                                                                    method='interpolate')),
                                                            ('trend_decomposer',
                                                             FunctionTransformer(check_inverse=False,
                                                                                 func=<function <lambda> at 0x7f725768fc70>)),
                                                            ('trend_forecaster',
                                                             NaiveForecaster())])),
                                ('seasonal_forecaster',
                                 ForecastingPipeline(steps=[('interpolator',
                                                             PandasTransformAdaptor(kwargs={'method': 'linear'},
                                                                                    method='interpolate')),
                                                            ('seasonal_decomposer',
                                                             FunctionTransformer(check_inverse=False,
                                                                                 func=<function <lambda> at 0x7f725768f910>)),
                                                            ('seasonal_forecaster',
                                                             NaiveForecaster(sp=365))]))],
                   regressor=SumRegressor())

With the definition of our model, we can finally build the whole pipeline!

In [18]:
# Creating Validation folds
nfolds = 5
forecast_size = 14
fh = np.arange(forecast_size) + 1
window_length = df.shape[0] - nfolds - forecast_size + 1
splitter = list(SlidingWindowSplitter(fh=fh, window_length=window_length).split(df))

# Iterate folds
metric = MeanAbsoluteScaledError()
train_score, val_score = 0, 0
bar = tqdm(splitter, desc="Cross-validating")
for train_idx, val_idx in bar:
    
    # Get train and validation data (interpolate for NaN in metric calculation)
    y_train, y_val = df.iloc[train_idx].interpolate(method="linear"), df.iloc[val_idx].interpolate(method="linear")
    
    # Get training performance
    forecaster = pipeline.fit(y_train.iloc[:-forecast_size], fh=fh)
    y_train_pred = pipeline.predict()
    train_score += metric(y_train.iloc[-forecast_size:], y_train_pred, y_train=y_train.iloc[:-forecast_size]) / len(splitter)
    
    # Predict on validation data
    forecaster = pipeline.fit(y_train, fh=fh)
    y_val_pred = pipeline.predict()
    val_score += metric(y_val, y_val_pred, y_train=y_train) / len(splitter)
    
print(f"Train score: {train_score} | Validation score: {val_score}")

Cross-validating:   4%|▍         | 2/50 [00:20<08:06, 10.14s/it]


KeyboardInterrupt: 